In [2]:
import vectorbt as vbt
import numpy as np
import pandas as pd

In [30]:
eth_usd = vbt.YFData.download(
    ['ETH-USD', 'BTC-USD'],
    missing_index='drop',
    interval='1m'
).get('Close')

/home/pratyush/.local/lib/python3.8/site-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.



In [31]:
def indicator(_closePrice, rsiWindow = 14, movingAverageWindow = 50, entry = 30, exit = 70):
    rsi = vbt.RSI.run(_closePrice, window= rsiWindow).rsi
    ma = vbt.MA.run(_closePrice, movingAverageWindow).ma
    trend = np.where(rsi > exit, -1, 0)
    trend = np.where( np.array(ma > _closePrice) & np.array(rsi < entry) , 1, trend )
    return trend

In [32]:
ind = vbt.IndicatorFactory(
    class_name="Combination",
    short_name="comb",
    input_names=['_closePrice'],
    param_names=['rsiWindow', 'movingAverageWindow', 'entry', 'exit'],
    output_names=['value']
).from_apply_func(indicator, rsiWindow = 14, movingAverageWindow = 50, entry=30, exit=70)

In [33]:
res = ind.run(_closePrice=eth_usd, 
            #   rsiWindow=np.arange(10, 40, step=3, dtype=int), 
            #   movingAverageWindow = np.arange(20, 200, step=15,dtype=int),
              entry = np.arange(10, 40, step=4, dtype=int),
              exit = np.arange(60, 85, step=4, dtype=int),
              param_product = True 
              )

In [34]:
exits = res.value == 1
entries = res.value == -1
pf = vbt.Portfolio.from_signals(eth_usd, entries=entries, exits=exits)

In [35]:

returns = pf.total_return()
returns.max()

0.045890081397262604

In [36]:
fig = returns.vbt.heatmap(x_level='comb_entry', y_level='comb_exit', slider_level='symbol')
fig

FigureWidget({
    'data': [{'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'hoverongaps': False,
              'hovertemplate': 'comb_entry: %{x}<br>comb_exit: %{y}<br>value: %{z}<extra></extra>',
              'name': 'BTC-USD',
              'type': 'heatmap',
              'uid': 'a9ae5708-337a-4570-82e3-9fbbbb4365d4',
              'visible': True,
              'x': array([10, 14, 18, 22, 26, 30, 34, 38]),
              'y': array([60, 64, 68, 72, 76, 80, 84]),
              'z': array([[-0.03228037, -0.03865866, -0.02879285, -0.00502016, -0.00010009,
        